In [ ]:
from module.ms_module import MS
from preprocess.ms_preprocess import MSIPreprocessor
from module.ms_data_manager_imzml import MSDataManagerImzML
from tools.plot import plot_spectrum
from logger import get_logger

logger = get_logger("example")

In [ ]:
# Run examples when executing this file directly
if __name__ == "__main__":
    FILE_PATH = "data/example.imzML"
    ms = MS()
    ms_md = MSDataManagerImzML(ms, filepath=FILE_PATH)
    ms_md.load_full_data_from_file()
    sp = ms[0]

    denoised = MSIPreprocessor.noise_reduction_spectrum(
        data=sp,
        method="wavelet",
        window=10,
    )
    plot_spectrum(denoised)

In [ ]:
# Plotting
plot_spectrum(
    base=sp,
    target=denoised,
    mz_range=(500.0, 510.0),
    intensity_range=(0, 1.2),
    title_suffix='test',
    overlay=False,
)
print(MSIPreprocessor.calculate_snr_spectrum(spectrum=denoised))


In [ ]:
for i, spectrum in enumerate(ms):
    denoised = MSIPreprocessor.noise_reduction_spectrum(spectrum,method="ma")
    ms[i].mz_list = denoised.mz_list
    ms[i].intensity = denoised.intensity

denoise_test = ms[0]
plot_spectrum(denoise_test,    
                mz_range=(500.0, 510.0),
                intensity_range=(0, 1.2),)


In [ ]:
for i, spectrum in enumerate(ms):
    peakpicked = MSIPreprocessor.peak_pick_spectrum(spectrum,relheight=0.001)
    ms[i].mz_list = peakpicked.mz_list
    ms[i].intensity = peakpicked.intensity


In [ ]:
peakpicked = ms[0]
plot_spectrum(peakpicked,
            plot_mode=["stem"])

In [ ]:
ms_ali = MSIPreprocessor.peak_alignment(ms_data=ms,ref_method='histogram',combiner='sum')

In [ ]:
spectrum_alit = ms_ali[0]
plot_spectrum(spectrum_alit,
            plot_mode=["stem"])

In [ ]:
ms_ali = MSIPreprocessor.peak_alignment(ms_data=ms,ref_method='mean',combiner='max',round_digits=4,half_window=5,snr_threshold=10,match_method='dp')

In [ ]:
spectrum_alit = ms_ali[0]
plot_spectrum(spectrum_alit,
            plot_mode=["stem"])

In [ ]:
ms_ali.plot_ms_mask()